In [7]:
import importlib
from app.core import chunking as ch
importlib.reload(ch)
from app.core.chunking import tokenize_and_chunk, validate_overlap
text = "Cümle 1. Cümle 2. Cümle 3. Cümle 4. Cümle 5. " * 40
chs = tokenize_and_chunk(text, max_tokens=120, overlap=30, min_chunk_tokens=20, store_tokens=True)
len(chs), validate_overlap(chs, 30)

(10, True)

In [6]:
from app.core.chunking import _get_tokenizer
tok = _get_tokenizer()
for a,b in zip(chs, chs[1:]):
    ta = tok.encode(a['text'])
    tb = tok.encode(b['text'])
    if len(ta) >=30 and len(tb) >=30 and ta[-30:] != tb[:30]:
        print('UYUMSUZ:', a['id'], b['id'])
        break

UYUMSUZ: c1 c2


In [9]:
long_sentence = "A " * 1200  # tek dev cümle

In [10]:
all(c['token_count'] <= 300 for c in chs)

True

In [11]:
max(c['token_count'] for c in chs)

300

In [12]:
chs2 = tokenize_and_chunk(long_sentence, max_tokens=300, overlap=50, min_chunk_tokens=20)
all(c['token_count'] <= 300 for c in chs2)

True

In [13]:
sample = "Bir. İki. Üç. Dört. Beş. " * 10
no_overlap = tokenize_and_chunk(sample, max_tokens=120, overlap=0)
with_overlap = tokenize_and_chunk(sample, max_tokens=120, overlap=40)
len(no_overlap), len(with_overlap)  # with_overlap genelde biraz daha fazla

(2, 2)

In [14]:
shorty = "A. B. C. D. E."
chs = tokenize_and_chunk(shorty, max_tokens=50, overlap=0, min_chunk_tokens=20)
# Sonuç 0 veya 1 küçük chunk (çoğu zaman 0) => min_chunk_tokens filtresi
chs

[]

In [15]:
from app.core.ingestion import basic_text_stats
from app.core.chunking import _get_tokenizer
txt = "Bu bir test cümlesidir. " * 40
stats = basic_text_stats(txt)
real_tokens = len(_get_tokenizer().encode(txt))
(stats['approx_tokens'], real_tokens, round(real_tokens / max(stats['approx_tokens'],1),2))

(208, 361, 1.74)

In [16]:
from app.core.chunking import tokenize_and_chunk
from app.core.embeddings import get_or_compute_embeddings
text = "Model farkı testi için örnek metin. " * 10
chs = tokenize_and_chunk(text)
e1 = get_or_compute_embeddings(chs, model='text-embedding-004')
e2 = get_or_compute_embeddings(chs, model='text-embedding-004-alt')
len(e1), len(e2)

(1, 1)

In [17]:
t1 = "A B C D E F G"
t2 = "A B C D E F G H"
chs1 = [{'id':'c1','text':t1,'token_count':1,'start_token':0,'end_token':0}]
chs2 = [{'id':'c1','text':t2,'token_count':1,'start_token':0,'end_token':0}]
from app.core.embeddings import _hash_key
_hash_key('text-embedding-004', t1) != _hash_key('text-embedding-004', t2)  # True

True

In [18]:
from app.core.embeddings import get_or_compute_embeddings
chs = [{'id':'c1','text':'Deterministik test','token_count':10,'start_token':0,'end_token':9}]
a = get_or_compute_embeddings(chs)[0]['embedding']
b = get_or_compute_embeddings(chs)[0]['embedding']
a == b  # True

True

In [19]:
from app.core.chunking import tokenize_and_chunk
from app.core.embeddings import get_or_compute_embeddings
text = "Test cümlesi. " * 50
chs = tokenize_and_chunk(text, max_tokens=200, overlap=30)
assert chs, 'Chunk üretilmedi'
for c in chs: assert c['token_count'] <= 200
emb = get_or_compute_embeddings(chs)
assert len(emb) == len(chs)
print('OK')

OK
